In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("example_data/Informe-Turismos-Enero-2024.pdf")
pages = loader.load_and_split()

print(pages[0])

page_content='Matriculaciones de Automóviles de turismo\nEnero 2024\n2024 2023 %24/23 2024 2023 %24/23\n68.685 64.038 7,3% 68.685 64.038 7,3%\nTop 10 Automóviles de turismo\nEnero-Enero\n1º TOYOTA 7.615 TOYOTA 7.615 DACIA SANDERO 2.233 DACIA SANDERO 2.233\n2º SEAT 5.073 SEAT 5.073 TOYOTA COROLLA 2.143 TOYOTA COROLLA 2.143\n3º KIA 4.594 KIA 4.594 MG ZS 1.626 MG ZS 1.626\n4º HYUNDAI 4.121 HYUNDAI 4.121 SEAT LEON 1.535 SEAT LEON 1.535\n5º PEUGEOT 3.811 PEUGEOT 3.811 TOYOTA YARIS CROSS 1.522 TOYOTA YARIS CROSS 1.522\n6º DACIA 3.780 DACIA 3.780 SEAT ARONA 1.501 SEAT ARONA 1.501\n7º BMW 3.593 BMW 3.593 HYUNDAI TUCSON 1.494 HYUNDAI TUCSON 1.494\n8º VOLKSWAGEN 3.405 VOLKSWAGEN 3.405 PEUGEOT 2008 1.465 PEUGEOT 2008 1.465\n9º MERCEDES 3.308 MERCEDES 3.308 SEAT IBIZA 1.371 SEAT IBIZA 1.371\n10º RENAULT 2.937 RENAULT 2.937 TOYOTA RAV 4 1.241 TOYOTA RAV 4 1.241\nAutomóviles de turismo: Detalle por carburante (Cuota)\nEne.      Feb. Mar.     Abr. May.     Jun. Jul.       Ago. Sep.     Oct.      Nov.

In [2]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [3]:
model_name = 'gpt-3.5-turbo-instruct'
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

Un prompt incompleto que produce una salida no correcta por una mala interpretación

In [36]:
from datetime import datetime
# Define your desired data structure.
class Turismo(BaseModel):
    anio_actual: int = Field(description="Año actual")
    valor_actual: int = Field(description="Número de ventas del año actual")
    anio_pasado: int = Field(description="Año anterior al actual")
    valor_pasado: int = Field(description="Número de ventas del año anterior")

    # You can add custom validation logic easily with Pydantic.
    @validator('anio_actual')
    def anio_is_actual(cls, field):
        if field != datetime.now().year:
            raise ValueError("No es el año actual")
        return field
    
    """
    # You can add custom validation logic easily with Pydantic.
    @validator('valor_actual')
    def anio_is_actual(cls, field):
        if field < 1000:
            raise ValueError("El valor es muy pequeño. Revisar formato")
        return field
    """
    
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Turismo)
    
template = """
        Te daré una página de documento acerca de ventas de turismos(automóviles). 
        Necesito que extraigas de dicho texto el número total de ventas del mes en cuestion para el año actual y el pasado.
        
        texto: {texto}
        \n{format_instructions}"""


prompt = PromptTemplate(
    template=template,
    input_variables=["texto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(texto=pages[0])

output = model.invoke(_input.to_string())

parser.parse(output)

Turismo(anio_actual=2024, valor_actual=68, anio_pasado=2023, valor_pasado=64)

## Solución 1 con fewshottemplate

In [37]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)


examples = [
    {"text": "67.326", "valor": "67326"},
    {"text": "58.568", "valor": "58568"}
    ]

example_template = """
Text: {text}
Valor: {valor}
"""

example_prompt = PromptTemplate(
    input_variables=["text", "valor"],
    template=example_template,
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=template,
    input_variables=["texto"],
    suffix="",
    example_separator="\n\n",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Create the LLMChain for the dynamic_prompt
chain = LLMChain(llm=llm, prompt=dynamic_prompt)

# Run the LLMChain with input_data
response = chain.invoke({"texto": pages[0].page_content})

print(response['text'])

{
  "anio_actual": 2024,
  "valor_actual": 67326,
  "anio_pasado": 2023,
  "valor_pasado": 58568
}


Suponiendo que evidentemente no sabemos que lo va a hacer que de error. Incluimos un nuevo validador y dará error

In [38]:
from datetime import datetime
# Define your desired data structure.
class Turismo(BaseModel):
    anio_actual: int = Field(description="Año actual")
    valor_actual: int = Field(description="Número de ventas del año actual")
    anio_pasado: int = Field(description="Año anterior al actual")
    valor_pasado: int = Field(description="Número de ventas del año anterior")

    # You can add custom validation logic easily with Pydantic.
    @validator('anio_actual')
    def anio_is_actual(cls, field):
        if field != datetime.now().year:
            raise ValueError("No es el año actual")
        return field
    
    
    # You can add custom validation logic easily with Pydantic.
    @validator('valor_actual')
    def anio_is_actual(cls, field):
        if field < 1000:
            raise ValueError("El valor es muy pequeño. Revisar formato")
        return field
    
    
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Turismo)
    
template = """
        Te daré una página de documento acerca de ventas de turismos(automóviles). 
        Necesito que extraigas de dicho texto el número total de ventas del mes en cuestion para el año actual y el pasado.
        
        texto: {texto}
        \n{format_instructions}"""


prompt = PromptTemplate(
    template=template,
    input_variables=["texto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(texto=pages[0])

output = model.invoke(_input.to_string())

parser.parse(output)


OutputParserException: Failed to parse Turismo from completion 
{
    "anio_actual": 2024,
    "valor_actual": 68.685,
    "anio_pasado": 2023,
    "valor_pasado": 64.038
}. Got: 1 validation error for Turismo
valor_actual
  El valor es muy pequeño. Revisar formato (type=value_error)

## Incluimos el rectificador automático

In [39]:
from langchain.output_parsers import OutputFixingParser

try:
    ans = parser.parse(output)
except Exception as e:
    print('intentando corregir salida..llamamos con el formato correcto nuevamente')
    o_parse = OutputFixingParser.from_llm(parser=parser, llm=model)
    ans = o_parse.parse(output)

ans

intentando corregir salida..llamamos con el formato correcto nuevamente


Turismo(anio_actual=2024, valor_actual=68685, anio_pasado=2023, valor_pasado=64038)